In [5]:
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [20]:
from loaders.loaders import *
from evaluation.split_dataset import split_on_seed_dataset
from query.query_functions2 import run_sparql_query, make_triplet, count_from_triples, run_sparql_query_paged

In [10]:
SEED = 44
SAMPLE_SIZE = 4

lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)
splitted_lcquad = split_on_seed_dataset(lc_quad_train, SAMPLE_SIZE, SEED)

In [11]:
for i, element in enumerate(splitted_lcquad):
    for triple in element["graph"]:
        if "religion" in triple[1]:
            print(i)

7
47
111
128
184
246


In [25]:
splitted_lcquad[184]

{'_id': '4573',
 'corrected_question': 'Which religions are followed by people in England?',
 'intermediary_question': 'List the <faith> of the <people> whose <official residence> is <England>.',
 'sparql_query': 'SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/residence> <http://dbpedia.org/resource/England> . ?x <http://dbpedia.org/property/religion> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Person>}',
 'sparql_template_id': 305,
 'result': ['Shia Islam',
  'http://dbpedia.org/resource/Islam',
  'Quaker',
  'http://dbpedia.org/resource/Christian',
  'http://dbpedia.org/resource/Atheism',
  'http://dbpedia.org/resource/Hindu',
  'http://dbpedia.org/resource/Catholic_Church',
  'http://dbpedia.org/resource/Anglicanism',
  'http://dbpedia.org/resource/Jews',
  'http://dbpedia.org/resource/Greek_Orthodox_Church',
  'Christianity',
  'Catholic',
  'The Church of Jesus Christ of Latter-day Saints',
  'Sunni Islam',
  'http://dbpe

In [15]:
count_from_triples(['?uri', '<http://dbpedia.org/property/religion>', '<http://dbpedia.org/resource/Catholic_Church>'])

10640

In [16]:
count_from_triples(['?uri', '<http://dbpedia.org/property/religion>', '<http://dbpedia.org/resource/Christianity>'])

985

In [27]:
from copy import deepcopy

def get_uri_triple(triples):
    valid_triples = list()
    for trip in triples:
        qmark_counter = 0
        for element in trip:
            if element[0] == "?":
                qmark_counter += 1
        if qmark_counter > 1:
            valid_triples.append(trip)
    return valid_triples

def replace_uri(trip, element):
    trip_copy = deepcopy(trip)
    for i, item in enumerate(trip):
        if item == "?uri":
            trip_copy[i] = element
    return trip_copy


def count_from_triples(triples):
    # query_command = f"SELECT count(?uri) WHERE {{ {' . '.join([make_triplet(item) for item in triples])} }}"
    query_command = f"SELECT count(?x) WHERE {{ {' . '.join([make_triplet(item) for item in triples]) if isinstance(triples[0], list) else make_triplet(triples)} }}"
    query_results = run_sparql_query(query_command)
    return int(query_results[0]['callret-0']['value'])

In [28]:
from tqdm import tqdm

res_size = list()

big_connection_list = list()

for record in tqdm(lc_quad_train):
    uri_triples = get_uri_triple(record["graph"])

    for trip in uri_triples:
        for gold in record["result_urlonly"]:
            runnable_trip = replace_uri(trip, gold)
            query_command = f"SELECT DISTINCT ?x WHERE {{ {' . '.join([make_triplet(item) for item in runnable_trip]) if isinstance(runnable_trip[0], list) else make_triplet(runnable_trip)} . }}"
            query_results = run_sparql_query_paged(query_command, 20)
            if query_results is None:
                continue

            if len(query_results) >= 20000:
                print(len(query_results))
                print(runnable_trip)
                print(count_from_triples(runnable_trip))
            res_size.append(len([item['x']['value'] for item in query_results]))

 23%|█████████▌                                | 57/250 [00:13<01:46,  1.81it/s]

200000
['?x', '<http://dbpedia.org/ontology/type>', 'http://dbpedia.org/resource/Privately_held_company']
11067


 25%|██████████▍                               | 62/250 [00:19<03:19,  1.06s/it]

200000
['?x', '<http://dbpedia.org/ontology/occupation>', 'http://dbpedia.org/resource/Actor']
19802


 26%|███████████                               | 66/250 [00:25<04:13,  1.38s/it]

...

 58%|███████████████████████▌                 | 144/250 [00:49<00:10,  9.80it/s]

200000
['?x', '<http://dbpedia.org/ontology/birthPlace>', 'http://dbpedia.org/resource/United_States']
32205


 73%|█████████████████████████████▊           | 182/250 [01:01<00:09,  7.07it/s]

200000
['?x', '<http://dbpedia.org/property/religion>', 'http://dbpedia.org/resource/Catholic_Church']
10640


100%|█████████████████████████████████████████| 250/250 [01:17<00:00,  3.24it/s]


In [30]:
runnable_trip = ['?x', '<http://dbpedia.org/ontology/birthPlace>', 'http://dbpedia.org/resource/United_States']

query_command = f"SELECT DISTINCT ?x WHERE {{ {' . '.join([make_triplet(item) for item in runnable_trip]) if isinstance(runnable_trip[0], list) else make_triplet(runnable_trip)} . }}"
query_results = run_sparql_query_paged(query_command, 100)

print(len(query_results))

1000000
